# Background Subtraction Method

This notebook is used to determine the most appropriate background subtraction method to use from the range of options available for the [Background2D](http://photutils.readthedocs.io/en/stable/photutils/background.html) function.

In [1]:
%load_ext autotime

from astropy.io import fits
import photutils
from photutils import Background2D, SigmaClip, MeanBackground, MMMBackground, \
    MedianBackground, SExtractorBackground, BkgIDWInterpolator, BkgZoomInterpolator

import numpy as np

from piaa import observation
from pocs.utils import current_time

# Load the RGB mask arrays and the camera bias
rgb_masks = np.load('/var/panoptes/rgb_masks.numpy')
camera_bias = 2048

### Fields to test

We test two separate fields, one on a new moon and relatively sparse in terms of stellar density, the other near the galactic equator and crowded

In [2]:
# Test a few different fields that have different conditions
image_dirs = {
    'crowded_field': '/var/panoptes/images/fields/Kic8462852/14d3bd/20170416T124816/',
    'new_moon': '/var/panoptes/images/fields/HatP20/14d3bd/20170128T053449/',
}

time: 1.45 ms


### Box and grid size

We test two different box sizes that are both integer multiples of the full image dimensions. The two sizes are complementary, so for the large box size of $316 \times 434$ a grid of $11 \times 12$ boxes are created. For the smaller box size the opposite is true.

In [3]:
# Image size: 3476 x 5208
box_size = [
    (316, 434),
    (11, 12)
]

time: 1.83 ms


### Options to test

In [4]:
estimators = [SExtractorBackground(), MedianBackground(), MeanBackground(), MMMBackground()]
interpolators = [BkgZoomInterpolator()]
sigmas = [3, 4, 5]
iters = [5, 7, 10]

time: 2.4 ms


### Perform test

Run through the various options and output the total time spent with each option

In [ ]:
for name, image_dir in image_dirs.items():
    obs = observation.Observation(image_dir, verbose=True)

    data = fits.getdata(obs.files[1]) - camera_bias
    
    print("{}".format(name))
    for h, w in box_size:
        print("\tBox: {} {}".format(h, w))
        for estimator in estimators:
            print("\t{}".format(estimator))
            for interpolator in interpolators:
                print("\t{}".format(interpolator))
                for sigma in sigmas:
                    for iterations in iters:
                        print("\tSigma: {} Iters: {}".format(sigma, iterations))
                        start_time = current_time()
                        for color, mask in zip(['R', 'G', 'B'], rgb_masks):
                            sigma_clip = SigmaClip(sigma=sigma, iters=iterations)
                            bkg = Background2D(data, (h, w), filter_size=(3, 3),
                                           sigma_clip=sigma_clip, bkg_estimator=estimator, mask=~mask,
                                           interpolator=interpolator)
                            end_time = current_time()

                            print("\t{} Background\t Value: {:.02f}\t RMS: {:.02f}".format(
                                color, bkg.background_median, bkg.background_rms_median))

                        print("\tTotal time: {:0.2f} seconds\n".format((end_time - start_time).sec))

********************************************************************************
Setting up Observation for analysis
crowded_field
	Box: 316 434
	Sigma: 3 Iters: 5
	R Background	 Value: 166.14	 RMS: 16.26
	G Background	 Value: 382.54	 RMS: 23.56
	B Background	 Value: 327.96	 RMS: 20.25
	Total time: 73.29 seconds

	Sigma: 3 Iters: 7
	R Background	 Value: 166.21	 RMS: 16.13
	G Background	 Value: 382.90	 RMS: 23.22
	B Background	 Value: 327.29	 RMS: 20.15
	Total time: 78.63 seconds

	Sigma: 3 Iters: 10
	R Background	 Value: 166.21	 RMS: 16.13
	G Background	 Value: 382.90	 RMS: 23.20
	B Background	 Value: 327.32	 RMS: 20.15
	Total time: 96.21 seconds

	Sigma: 4 Iters: 5
	R Background	 Value: 164.86	 RMS: 18.63
	G Background	 Value: 380.38	 RMS: 27.73
	B Background	 Value: 327.04	 RMS: 22.73
	Total time: 66.92 seconds

	Sigma: 4 Iters: 7
	R Background	 Value: 164.96	 RMS: 18.42
	G Background	 Value: 380.93	 RMS: 27.17
	B Background	 Value: 327.20	 RMS: 22.55
	Total time: 83.66 seconds

	Sig